In [18]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import torch.nn as nn
import numpy as np
from math import exp

In [19]:
def gaussian(window_size, sigma):
  gauss = torch.Tensor([exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
  return gauss/gauss.sum()

In [20]:
def create_window(window_size, channel):
  _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
  _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)

  window = Variable(_2D_window.expand(channel, 1, window_size, window_size).contiguous())

  return window

In [21]:
def ssim(img1, img2, window, window_size, channel, size_average=True):
  mu1 = F.conv2d(img1, window, padding=window_size//2, groups=channel)  # μ(x)
  mu2 = F.conv2d(img2, window, padding=window_size//2, groups=channel)  # μ(y)

  mu1_sq = mu1.pow(2)                                                   # μ(x)^2
  mu2_sq = mu2.pow(2)                                                   # μ(y)^2
  mu1_mu2 = mu1*mu2                                                     # μ(xy)

  sigma1_sq = F.conv2d(img1*img1, window, padding=window_size//2, groups=channel) - mu1_sq    # σ(x)

  sigma2_sq = F.conv2d(img2*img2, window, padding=window_size//2, groups=channel) - mu2_sq    # σ(y)

  sigma12 = F.conv2d(img1*img2, window, padding=window_size//2, groups=channel) - mu1_mu2        # σ(xy)
  
  C1 = 0.01**2
  C2 = 0.03**2

  ssim_map = ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*(sigma1_sq + sigma2_sq + C2))

  if size_average:
        return ssim_map.mean()
  else:
        return ssim_map.mean(1).mean(1).mean(1)

In [22]:
class SSIM(torch.nn.Module):
  def __init__(self, window_size=11, size_average=True):
    super(SSIM, self).__init__()

    self.window_size = window_size
    self.size_average = size_average
    self.channel = 1
    self.window = create_window(window_size, self.channel)

  def forward(self, img1, img2):
    (_, channel, _, _) = img1.size()
    
    if channel == self.channel and self.window.data.type() == img1.data.type():
       window = self.window
    else:
       window = create_window(self.window_size, channel)

       window = window.type_as(img1)

       self.window = window
       self.channel = channel

    return ssim(img1, img2, window, self.window_size, channel, self.size_average)